<a href="https://colab.research.google.com/github/lialic/datasciencecoursera/blob/master/Python/SVM_Hyper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM and tune params with GridSearch

MSSV: 19520208
Ten: Huynh Phuong Nhu

## Import libs 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC

SVM có những siêu tham số gì? Miền giá trị của những siêu tham số

C : float, optional (default=1.0)
    Penalty parameter C of the error term.

degree : int, optional (default=3)
   Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.


tol : float, optional (default=1e-3)
   Tolerance for stopping criterion.

```
param_grid = [
 {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
 {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Data/CS116/Social_Network_Ads.csv'
df = pd.read_csv(path)
df.head

<bound method NDFrame.head of      Age  EstimatedSalary  Purchased
0     19            19000          0
1     35            20000          0
2     26            43000          0
3     27            57000          0
4     19            76000          0
..   ...              ...        ...
395   46            41000          1
396   51            23000          1
397   50            20000          1
398   36            33000          0
399   49            36000          1

[400 rows x 3 columns]>

In [ ]:
x, y = df.iloc[:, :-1], df.iloc[:, -1]

## Split train:val:test to be 6:2:2

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

## Normalize the data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_new = scaler.fit_transform(x_train)

## Find the best hyper-params and train using GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C': [0.1, 0.5, 1, 10], 'gamma': [0.5, 0.1, 0.005 ,0.001, 0.0001], 'degree': [3, 5, 7, 9]}
best_acc = 0

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
model = SVC()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x_new, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.920833 using {'C': 0.5, 'degree': 3, 'gamma': 0.5, 'kernel': 'rbf'}
0.833333 (0.045166) with: {'C': 0.1, 'degree': 3, 'gamma': 0.5, 'kernel': 'linear'}
0.914583 (0.041090) with: {'C': 0.1, 'degree': 3, 'gamma': 0.5, 'kernel': 'rbf'}
0.804167 (0.027873) with: {'C': 0.1, 'degree': 3, 'gamma': 0.5, 'kernel': 'poly'}
0.764583 (0.043626) with: {'C': 0.1, 'degree': 3, 'gamma': 0.5, 'kernel': 'sigmoid'}
0.833333 (0.045166) with: {'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'linear'}
0.850000 (0.041063) with: {'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf'}
0.653125 (0.006250) with: {'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}
0.797917 (0.037180) with: {'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid'}
0.833333 (0.045166) with: {'C': 0.1, 'degree': 3, 'gamma': 0.005, 'kernel': 'linear'}
0.653125 (0.006250) with: {'C': 0.1, 'degree': 3, 'gamma': 0.005, 'kernel': 'rbf'}
0.653125 (0.006250) with: {'C': 0.1, 'degree': 3, 'gamma': 0.005, 'kernel': 'poly'}
0.65312

## Normalize the test set and test

In [ ]:
x_test_new = scaler.transform(x_test)
pred = grid_search.predict(x_test_new)
print(accuracy_score(y_test, pred) )
print(classification_report(y_test, pred))

0.8625
              precision    recall  f1-score   support

           0       0.95      0.81      0.88        48
           1       0.77      0.94      0.85        32

    accuracy                           0.86        80
   macro avg       0.86      0.88      0.86        80
weighted avg       0.88      0.86      0.86        80

